In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sn
import scipy.stats as stats

In [2]:
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Imported the necessary libraries

In [3]:
columns = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label", "last_flag"]

Defined the columns

In [4]:
train = pd.read_csv('C:/Users/HP/Downloads/3. Network Intrusion Detection System/NSL_Dataset/Train.txt' , 
                      sep = "," , names = columns)
train = train.iloc[:,:-1]

In [5]:
test = pd.read_csv('C:/Users/HP/Downloads/3. Network Intrusion Detection System/NSL_Dataset/Test.txt' , 
                      sep = "," , names = columns)
test = test.iloc[:,:-1]

imported the datasets

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 42 columns):
duration                       125973 non-null int64
protocol_type                  125973 non-null object
service                        125973 non-null object
flag                           125973 non-null object
src_bytes                      125973 non-null int64
dst_bytes                      125973 non-null int64
land                           125973 non-null int64
wrong_fragment                 125973 non-null int64
urgent                         125973 non-null int64
hot                            125973 non-null int64
num_failed_logins              125973 non-null int64
logged_in                      125973 non-null int64
num_compromised                125973 non-null int64
root_shell                     125973 non-null int64
su_attempted                   125973 non-null int64
num_root                       125973 non-null int64
num_file_creations             125973 

In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22544 entries, 0 to 22543
Data columns (total 42 columns):
duration                       22544 non-null int64
protocol_type                  22544 non-null object
service                        22544 non-null object
flag                           22544 non-null object
src_bytes                      22544 non-null int64
dst_bytes                      22544 non-null int64
land                           22544 non-null int64
wrong_fragment                 22544 non-null int64
urgent                         22544 non-null int64
hot                            22544 non-null int64
num_failed_logins              22544 non-null int64
logged_in                      22544 non-null int64
num_compromised                22544 non-null int64
root_shell                     22544 non-null int64
su_attempted                   22544 non-null int64
num_root                       22544 non-null int64
num_file_creations             22544 non-null int64
num_

In [8]:
#Separating the categorical columns

cat_var = train[['protocol_type', 'service', 'flag']]

In [9]:
cat_var.columns

Index(['protocol_type', 'service', 'flag'], dtype='object')

In [10]:
def create_dummies(df, colname):
    col_dummies = pd.get_dummies(df[colname], prefix = colname, drop_first = True)
    df = pd.concat([df, col_dummies], axis = 1)
    df.drop(colname, axis = 1, inplace = True )
    return df

In [11]:
for c_feature in ['protocol_type', 'service', 'flag']:
    cat_var[c_feature] = cat_var[c_feature].astype('category')
    cat_var = create_dummies(cat_var, c_feature)

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


created dummies for categorical columns

In [12]:
cat_var.columns

Index(['protocol_type_tcp', 'protocol_type_udp', 'service_X11',
       'service_Z39_50', 'service_aol', 'service_auth', 'service_bgp',
       'service_courier', 'service_csnet_ns', 'service_ctf', 'service_daytime',
       'service_discard', 'service_domain', 'service_domain_u', 'service_echo',
       'service_eco_i', 'service_ecr_i', 'service_efs', 'service_exec',
       'service_finger', 'service_ftp', 'service_ftp_data', 'service_gopher',
       'service_harvest', 'service_hostnames', 'service_http',
       'service_http_2784', 'service_http_443', 'service_http_8001',
       'service_imap4', 'service_iso_tsap', 'service_klogin', 'service_kshell',
       'service_ldap', 'service_link', 'service_login', 'service_mtp',
       'service_name', 'service_netbios_dgm', 'service_netbios_ns',
       'service_netbios_ssn', 'service_netstat', 'service_nnsp',
       'service_nntp', 'service_ntp_u', 'service_other', 'service_pm_dump',
       'service_pop_2', 'service_pop_3', 'service_printer', 'se

In [13]:
cat_var_test = test[['protocol_type', 'service', 'flag']]

In [14]:
for c_feature in ['protocol_type', 'service', 'flag']:
    cat_var_test[c_feature] =cat_var_test[c_feature].astype('category')
    cat_var_test = create_dummies(cat_var_test, c_feature)

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


repeated the process for test data

In [15]:
cat_var_test.columns

Index(['protocol_type_tcp', 'protocol_type_udp', 'service_X11',
       'service_Z39_50', 'service_auth', 'service_bgp', 'service_courier',
       'service_csnet_ns', 'service_ctf', 'service_daytime', 'service_discard',
       'service_domain', 'service_domain_u', 'service_echo', 'service_eco_i',
       'service_ecr_i', 'service_efs', 'service_exec', 'service_finger',
       'service_ftp', 'service_ftp_data', 'service_gopher',
       'service_hostnames', 'service_http', 'service_http_443',
       'service_imap4', 'service_iso_tsap', 'service_klogin', 'service_kshell',
       'service_ldap', 'service_link', 'service_login', 'service_mtp',
       'service_name', 'service_netbios_dgm', 'service_netbios_ns',
       'service_netbios_ssn', 'service_netstat', 'service_nnsp',
       'service_nntp', 'service_ntp_u', 'service_other', 'service_pm_dump',
       'service_pop_2', 'service_pop_3', 'service_printer', 'service_private',
       'service_remote_job', 'service_rje', 'service_shell', 'servi

In [16]:
trainservice = train['service'].tolist()

testservice = test['service'].tolist()

difference = list(set(trainservice) - set(testservice))

string = 'service_'

difference = [string + x for x in difference]

difference

['service_aol',
 'service_http_8001',
 'service_http_2784',
 'service_harvest',
 'service_red_i',
 'service_urh_i']

In [17]:
for x in difference:
    cat_var_test[x] = 0

cat_var_test.shape

(22544, 81)

In [18]:
new_train = train.join(cat_var)
new_train.drop('flag', axis=1, inplace=True)
new_train.drop('protocol_type', axis=1, inplace=True)
new_train.drop('service', axis=1, inplace=True)


new_test=test.join(cat_var_test)
new_test.drop('flag', axis=1, inplace=True)
new_test.drop('protocol_type', axis=1, inplace=True)
new_test.drop('service', axis=1, inplace=True)

print(new_train.shape)
print(new_test.shape)

(125973, 120)
(22544, 120)


merged the categprical columns with original train data

dropped flag , protocol_type , service columns because already made dummies for them

repeated for test data as well

In [19]:
label_train = new_train['label']
label_test = new_test['label']

In [20]:
newlabel_train = np.where(label_train == 'normal' , 0 , 1)

#labelling 'normal' as 0 & attack as 1

In [21]:
newlabel_test = np.where(label_test == 'normal' , 0 , 1)

#labelling 'normal' as 0 & attack as 1

In [22]:
new_train['label'] = newlabel_train
new_test['label'] = newlabel_test

In [23]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [24]:
feature_columns = new_train.columns.difference(['label'])

#defining the feature columns

In [25]:
logreg = LogisticRegression(solver='lbfgs')
rfe = RFE(logreg, n_features_to_select = 20, verbose = 10)

In [26]:
new_train['label'] = new_train['label'].astype('int')

In [27]:
rfe = rfe.fit(new_train[feature_columns], new_train.label)

Fitting estimator with 119 features.


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Fitting estimator with 118 features.


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Fitting estimator with 117 features.


...

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Fitting estimator with 21 features.


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Selected important feature columns with the help of RFE

In [28]:
print (rfe.support_)
print (rfe.ranking_)

[False  True False False  True False  True False False False False False
  True False False False  True  True False  True  True False False False
 False  True False False False False False False False False False False
 False  True False False  True False False False False False False False
 False False False False False False False  True  True False False False
 False False  True False False False False False False  True False False
 False False False False False False False False False False False  True
 False False False False False False  True False False False False False
 False False False False False False False False False False  True False
 False False False False False False False  True False False  True]
[ 74   1  92  86   1  44   1  68  70  77   3  66   1  91  64   4   1   1
  47   1   1  35  73  72  63   1  93   2  54  50   8  36  57 100   7  56
  71   1  88  39   1  67  59  11  97  60  16  13  20  10  42  43  94  52
  41   1   1  22  65  58  55  62   1  96  19  49  99  12

In [29]:
X_final = new_train[feature_columns[rfe.support_]] #X = columns found by RFE
y_final = new_train['label'] #Y - label

In [30]:
#splitting into train & test

from sklearn.model_selection import train_test_split


train_X, test_X, train_y, test_y = train_test_split(X_final, y_final, test_size = 0.3, random_state = 12345)

In [31]:
from sklearn.tree import DecisionTreeClassifier
clf1 = DecisionTreeClassifier(random_state=0)

#trying decision tree model first

In [32]:
clf1.fit(train_X, train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [33]:
acc_dt = round( clf1.score(train_X, train_y) * 100, 2)
print (str(acc_dt) + ' percent')

98.31 percent


Decison tree model seems to be overfitting therefore trying Random Forest next

In [34]:
from sklearn.ensemble import RandomForestRegressor

clf2 = RandomForestRegressor(n_estimators=100)
clf2.fit(train_X, train_y)

acc_rf= round(clf2.score(train_X, train_y) * 100, 2)
print ("Accuracy: %i %% \n"%acc_rf)

Accuracy: 94 % 



Random Forest seems to be appropriate , predicting for test data with random forest

In [35]:
new_test1 = new_test[['diff_srv_rate', 'dst_host_diff_srv_rate',
       'dst_host_same_src_port_rate', 'dst_host_same_srv_rate',
       'dst_host_serror_rate', 'dst_host_srv_diff_host_rate', 'flag_RSTR',
       'flag_S0', 'logged_in', 'protocol_type_tcp', 'same_srv_rate',
       'service_eco_i', 'service_ecr_i', 'service_http', 'service_private',
       'service_smtp', 'service_telnet', 'srv_rerror_rate', 'srv_serror_rate',
       'wrong_fragment']]

In [36]:
clf2.predict(new_test1)

array([0.71, 0.8 , 0.  , ..., 0.  , 0.  , 0.64])

In [38]:
Y_pred=clf2.predict(new_test1)

In [39]:
# Creating confusion matrix

pd.crosstab(new_test.label , Y_pred, rownames=['Actual attacks'], colnames=['Predicted attacks']).T

Actual attacks,0,1
Predicted attacks,,
0.000000,7118,3624
0.000074,0,6
0.000192,0,2
0.000236,1,45
0.000336,1,1
0.000402,1,1
0.000715,0,2
0.000748,2,3
0.000763,0,3
